# In this analysis, I will be utilizing the Foursquare API as well as Folium to perform clustering of Neighborhoods in Toronto. 
### First, I will need to scrape the neighborhood information from Wikipedia and turn it into a pandas Dataframe

In [1]:
#Installing our dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
data = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

In [3]:
#Since read_html produces a LIST of dataframes, we must index in order to capture the dataframe we are looking for.
data = data[0]
data.head()
print(data.shape)

(288, 3)


In [4]:
#Drop the Neighborhoods with no assigned Borough
data = data[data['Borough'] != 'Not assigned']

In [5]:
#Shape of data after dropping the rows with no Borough assigned
data.shape

(211, 3)

In [6]:
data.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [7]:
#Use comma as seperator and join all neighborhoods in a particular group or postcode in this example
data = data.groupby(['Postcode','Borough']).apply(lambda group: ','.join(group['Neighbourhood'])).reset_index()

In [8]:
data.columns = ['Postcode','Borough','Neighborhood']
data.head()

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [9]:
data.shape

(103, 3)

In [10]:
#Check which Boroughs have a Neighborhod with a value Not assigned
data[data['Neighborhood'] == 'Not assigned']

,Postcode,Borough,Neighborhood
85,M7A,Queen's Park,Not assigned


In [11]:
#Replace the Neighborhood value with the value of the Borough Column
data.iloc[85].Neighborhood = "Queen's Park"

In [12]:
#Double Checking that column was changed
data.iloc[85]

Postcode                 M7A
Borough         Queen's Park
Neighborhood    Queen's Park
Name: 85, dtype: object

In [13]:
#One more search to make sure no columns have an unassigned Neighborhood
data[data['Neighborhood'] == 'Not assigned']

,Postcode,Borough,Neighborhood


In [14]:
#Final Data Shape
data.shape

(103, 3)

# Part 2: Adding latitude/longitude info to Dataframe

In [15]:
location = pd.read_csv('Geospatial_Coordinates.csv')

In [16]:
location.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [17]:
data.rename(columns= {'Postcode':'Postal Code'},inplace=True)

In [18]:
#Concat means to simply glue together
#Merge means to bring together based on a common column
data = data.merge(location)

In [19]:
print(data.isnull().sum())
print(data.shape)

Postal Code     0
Borough         0
Neighborhood    0
Latitude        0
Longitude       0
dtype: int64
(103, 5)


In [23]:
data['Borough'].value_counts()

North York          24
Downtown Toronto    18
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
East York            5
East Toronto         5
York                 5
Queen's Park         1
Mississauga          1
Name: Borough, dtype: int64

In [28]:
#Pull a dataframe out that has the string Toronto within the borough columns. Not case sensitive
Toronto = data[data.Borough.str.contains('Toronto',case=False)]

In [29]:
Toronto

,Postal Code,Borough,Neighborhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
45,M4P,Central Toronto,Davisville North,43.712751,-79.390197
46,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
47,M4S,Central Toronto,Davisville,43.704324,-79.388790
48,M4T,Central Toronto,"Moore Park,Summerhill East",43.689574,-79.383160
49,M4V,Central Toronto,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",43.686412,-79.400049


# Part 3: Analysis of geographical info. Including Clustering techniques

In [36]:
import geopy
from geopy import Nominatim
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="to_explorer") #Must create nominatim with user_agent
location = geolocator.geocode(address) #Use geocode function on addres to create object with latitude and longitude
Tolatitude = location.latitude 
Tolongitude = location.longitude
print('The geograpical coordinate of Toronto, Ontario are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto, Ontario are 43.653963, -79.387207.


In [38]:
import folium

In [48]:
#Create a map starting at Toronto
map_toronto = folium.Map(location=[Tolatitude,Tolongitude])

In [49]:
map_toronto

In [50]:
for Borough, Neighborhood, Latitude, Longitude in zip(Toronto['Borough'], Toronto['Neighborhood'], Toronto['Latitude'], Toronto['Longitude']):
    label = '{},{}'.format(Neighborhood, Borough)
    label = folium.Popup(label)
    folium.CircleMarker(
        location=[Latitude,Longitude],
        radius=5,
        popup=label,
        color='blue',
        parsehtml=False).add_to(map_toronto)
map_toronto

In [53]:
Toronto.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38 entries, 37 to 87
Data columns (total 5 columns):
Postal Code     38 non-null object
Borough         38 non-null object
Neighborhood    38 non-null object
Latitude        38 non-null float64
Longitude       38 non-null float64
dtypes: float64(2), object(3)
memory usage: 1.8+ KB


In [54]:
#Now we will utilize Foursquare to obtain data to segment the neighborhoods into clusters.
CLIENT_ID = 'XU5HNSBK44R1O2ITYVXND32VRNVZFSNUSMO2WMPPIIPJOBSU' # your Foursquare ID
CLIENT_SECRET = 'FA0053DZG5N54N2KNUBDMY4T0I4ZV0WEOPKPRJH0XHTW32OY' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [56]:
Toronto.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [62]:
#Lets try foursquare out with one neighborhood!
Neighborhood1 = Toronto.iloc[0,2]
Lat1 = Toronto.iloc[0,3]
Long1 = Toronto.iloc[0,4]
print(Neighborhood1)
print(Lat1)
print(Long1)

The Beaches
43.67635739999999
-79.2930312


In [63]:
#The url search query will get us up to 100 venues within a 500 meter radius of The Beaches
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID,CLIENT_SECRET,Lat1,Long1, VERSION, 500, 100)


In [65]:
#Here is the json file 
import requests
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d82ac9f86bc49003982b10e'},
 'response': {'headerLocation': 'The Beaches',
  'headerFullLocation': 'The Beaches, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 4,
  'suggestedBounds': {'ne': {'lat': 43.680857404499996,
    'lng': -79.28682091449052},
   'sw': {'lat': 43.67185739549999, 'lng': -79.29924148550948}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bd461bc77b29c74a07d9282',
       'name': 'Glen Manor Ravine',
       'location': {'address': 'Glen Manor',
        'crossStreet': 'Queen St.',
        'lat': 43.67682094413784,
        'lng': -79.29394208780985,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.67682094413784,
          'lng': -79.29394208780985}],
        'distanc

In [66]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories'] #Go through each row and try this key
    except:
        categories_list = row['venue.categories'] #If the above key does not work, try this one.
        
    if len(categories_list) == 0: #If there is no category, return none.
        return None
    else:
        return categories_list[0]['name'] #Return the name of the categoy

In [83]:
#First go to response, then go to groups returned by the request, 
venues = results['response']['groups'][0]['items']

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

nearby_venues = json_normalize(venues) #Returns a mess of a dataframe containing the info from the Json file

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,categories,lat,lng
0,Glen Manor Ravine,Trail,43.676821,-79.293942
1,The Big Carrot Natural Food Market,Health Food Store,43.678879,-79.297734
2,Grover Pub and Grub,Pub,43.679181,-79.297215
3,Upper Beaches,Neighborhood,43.680563,-79.292869


In [88]:
#This function will be used to grab all the nearby venues for each neighborhood

LIMIT=100
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [89]:
Toronto_venues = getNearbyVenues(names=Toronto['Neighborhood'],latitudes=Toronto['Latitude'],longitudes=Toronto['Longitude'])

The Beaches
The Danforth West,Riverdale
The Beaches West,India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park,Summerhill East
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West
Rosedale
Cabbagetown,St. James Town
Church and Wellesley
Harbourfront,Regent Park
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide,King,Richmond
Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Roselawn
Forest Hill North,Forest Hill West
The Annex,North Midtown,Yorkville
Harbord,University of Toronto
Chinatown,Grange Park,Kensington Market
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place,Underground city
Christie
Dovercourt Village,Dufferin
Little Portugal,Trinity
Brockton,Exhibition Place,Parkdale Village
High Park,The Junction South
Parkdale,Roncesvall

In [94]:
print('Foursquare has returned {} venues for all the neighborhoods in Toronto'.format(Toronto_venues.shape[0]))

Foursquare has returned 1702 venues for all the neighborhoods in Toronto


In [119]:
#We will now start to analyze each neighborhood. Starting by looking at the venue types for each neighborhood
onehot_venues = pd.get_dummies(Toronto_venues['Venue Category'])
onehot_venues['Neighborhood'] = Toronto_venues['Neighborhood']


In [122]:
onehot_venues.shape

(1702, 239)

In [126]:
Toronto_grouped = onehot_venues.groupby('Neighborhood').mean().reset_index()

In [127]:
Toronto_grouped

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,"Adelaide,King,Richmond",0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.030000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.020000,0.000000,0.000000,0.010000,0.000000,0.000000
1,Berczy Park,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.018182,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Brockton,Exhibition Place,Parkdale Village",0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.045455
3,Business Reply Mail Processing Centre 969 Eastern,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",0.000000,0.0625,0.0625,0.0625,0.125,0.125,0.125,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,"Cabbagetown,St. James Town",0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Central Bay Street,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.011905,0.000000,...,0.000000,0.000000,0.000000,0.00,0.011905,0.000000,0.000000,0.011905,0.000000,0.011905
7,"Chinatown,Grange Park,Kensington Market",0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.010000,0.000000,0.00,0.060000,0.000000,0.040000,0.010000,0.000000,0.000000
8,Christie,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Church and Wellesley,0.011111,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.011111,0.000000,...,0.011111,0.000000,0.000000,0.00,0.000000,0.011111,0.011111,0.000000,0.011111,0.011111


In [128]:
Toronto_grouped.shape

(38, 239)

In [129]:
num_top_venues = 5

for hood in Toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
                 venue  freq
0          Coffee Shop  0.08
1                 Café  0.05
2           Steakhouse  0.04
3                  Bar  0.04
4  American Restaurant  0.03


----Berczy Park----
          venue  freq
0   Coffee Shop  0.07
1  Cocktail Bar  0.05
2          Café  0.04
3   Cheese Shop  0.04
4      Beer Bar  0.04


----Brockton,Exhibition Place,Parkdale Village----
            venue  freq
0  Breakfast Spot  0.09
1     Coffee Shop  0.09
2            Café  0.09
3          Bakery  0.05
4       Pet Store  0.05


----Business Reply Mail Processing Centre 969 Eastern----
                venue  freq
0  Light Rail Station  0.12
1             Brewery  0.06
2          Skate Park  0.06
3       Garden Center  0.06
4              Garden  0.06


----CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara----
              venue  freq
0    Airport Lounge  0.12
1   Airport Service  0.12
2  Airport Terminal  0.12
3

In [142]:
#Now we will begin forming our cluster analysis.
Toronto_cluster = Toronto_grouped.drop('Neighborhood',axis=1)

# import k-means from clustering stage
from sklearn.cluster import KMeans

Cluster = KMeans(n_clusters=5).fit(Toronto_cluster)
Cluster.labels_

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1,
       4, 1, 0, 1, 1, 4, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [143]:
Toronto_grouped.insert(0, 'Cluster Labels', Cluster.labels_)

In [190]:
Cluster_groups = Toronto_grouped.groupby('Cluster Labels').sum().T
Cluster_groups.head()

Cluster Labels,0,1,2,3,4
Afghan Restaurant,0.0,0.011111,0.0,0.0,0.0
Airport,0.0,0.062500,0.0,0.0,0.0
Airport Food Court,0.0,0.062500,0.0,0.0,0.0
Airport Gate,0.0,0.062500,0.0,0.0,0.0
Airport Lounge,0.0,0.125000,0.0,0.0,0.0


In [183]:
#Top 5 Areas for cluster group 0
Cluster_groups[0].nlargest(n=5)

Park                  0.5
Tennis Court          0.5
Afghan Restaurant     0.0
Airport               0.0
Airport Food Court    0.0
Name: 0, dtype: float64

In [184]:
#Top 5 Areas for cluster group 1
Cluster_groups[1].nlargest(n=5)

Coffee Shop           2.385839
Café                  1.792187
Restaurant            0.923279
Italian Restaurant    0.882765
Park                  0.854568
Name: 1, dtype: float64

In [185]:
#Top 5 Areas for cluster group 2
Cluster_groups[2].nlargest(n=5)

Garden               0.333333
Home Service         0.333333
Pool                 0.333333
Afghan Restaurant    0.000000
Airport              0.000000
Name: 2, dtype: float64

In [186]:
#Top 5 Areas for cluster group 3
Cluster_groups[3].nlargest(n=5)

Jewelry Store         0.25
Mexican Restaurant    0.25
Sushi Restaurant      0.25
Trail                 0.25
Afghan Restaurant     0.00
Name: 3, dtype: float64

In [187]:
#Top 5 Areas for cluster group 4
Cluster_groups[4].nlargest(n=5)

Park           0.733333
Bus Line       0.333333
Swim School    0.333333
Building       0.200000
Playground     0.200000
Name: 4, dtype: float64

### From the above cluster analysis, it would appear that:

Cluster 0 is an outdoor/active type of area.

Cluster 1 and Cluster 3 are more urban areas.

Cluster 2 could be around a shopping center focused on homegoods.

Cluster 4 is an area for academia. 